In [1]:
# Reads in stellar spectrum, packs it on top of a 2D array to make it seem like it's real

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pickle
from astropy.io import fits
from scipy.ndimage import gaussian_filter

In [2]:
stem_spec = '/Users/bandari/Documents/git.repos/rrlfe/src/model_spectra/rrmods_all/original_ascii_files'

In [3]:
spec_fake = pd.read_csv(stem_spec + '/700020m30.smo', delim_whitespace=True, names=['wavel','flux','noise'])

In [4]:
# normalize flux somehow
norm_val = 1.
spec_fake['flux_norm'] = norm_val*np.divide(spec_fake['flux'],np.max(spec_fake['flux']))

In [6]:
# make cubes of command-response pairs

N_cmd = 100
step_wavel = int(1000/N_cmd) # step size of impulse on detector, in pixels
x_detec_size = 38
y_detec_size = 1000

N_pixels = x_detec_size*y_detec_size
cube_w_step_impulse = np.zeros((100,x_detec_size,y_detec_size))
cube_w_commands = np.zeros((N_cmd,N_cmd))

M_basis_set_cmds = N_cmd
poke_matrix = np.zeros((N_pixels,M_basis_set_cmds))

for t in range(0,N_cmd):
    
    vec_command = np.zeros(N_cmd)
    
    # update command vector
    vec_command[t] = 1
    
    # make the fake responses
    blank_2d = np.zeros((x_detec_size,1000))
    spec_perfect = blank_2d
   
    # a white light spectrum
    spec_perfect[int(0.5*y_detec_size),t*step_wavel:(t+1)*step_wavel] = norm_val*np.ones(len(spec_fake['flux_norm']))[t*step_wavel:(t+1)*step_wavel]

    blank_2d = np.zeros((21,1000)) #np.shape(flux_2d_perfect)
    spec_perfect = blank_2d
    spec_perfect[int(0.5*y_detec_size),:] = norm_val*np.ones(len(spec_fake['flux_norm']))
    
    spec_perfect[:,:t*(int(y_detec_size/N_cmd))] = 0
    spec_perfect[:,(t+1)*int(y_detec_size/N_cmd):] = 0
    
    spec_convolved = gaussian_filter(spec_perfect, sigma=5)

    test_rotate = scipy.ndimage.rotate(spec_convolved, angle=1, reshape=False)

    plt.imshow(test_rotate)
    plt.show()
    
    '''
    # noisy background
    array_2d_substrate = 0.01*np.random.normal(size=(x_detec_size,y_detec_size))
    '''
    # no noise
    #array_2d_substrate = np.zeros((x_detec_size,y_detec_size))
    
    array_2d_w_spec = test_rotate
    
    test_response = np.copy(array_2d_w_spec) # cosmetic
    
    # accumulate flattened responses
    flattened = test_response.flatten()

    # each column is one 'poke', and the elements are the response function
    poke_matrix[:,t] = flattened
    #poke_matrix[i,:] = flattened
    
    '''
    plt.imshow(array_2d_w_spec)
    plt.show()
    '''

    # add commands to cube
    cube_w_commands[t,:] = vec_command
    
    # add frame to cube
    cube_w_step_impulse[t,:,:] = array_2d_w_spec

IndexError: index 500 is out of bounds for axis 0 with size 38

In [6]:
# pseudoinverse: the instrument response matrix
response_matrix = np.linalg.pinv(poke_matrix)

In [16]:
# option to write to FITS
'''
hdu = fits.PrimaryHDU(cube_w_step_impulse)
hdul = fits.HDUList([hdu])
hdul.writeto('junk_poke_white_light_no_noise_small_footprint.fits')
'''

In [7]:
test_response = cube_w_step_impulse
test_cmds = cube_w_commands
response_matrix = response_matrix

# option to pickle
data_list = [response_matrix, test_cmds, test_response]
file_name = "junk_white_light_no_noise_small_footprint.pkl"
open_file = open(file_name, "wb")
pickle.dump(data_list, open_file)
open_file.close()